### Web scraping module

In [1]:
!pip3 install requests bs4

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [20]:
scrape_and_save_json("https://www.efsa.europa.eu/en/efsajournal/pub/2980")

✅ Saved JSON knowledge base for Efsa.Europa.Eu → knowledge_base/efsa.europa.eu.json
